In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO
from ultralytics.yolo.v8.detect.predict import DetectionPredictor
from ultralytics.yolo.utils.plotting import Annotator
import os
import supervision as sv

In [ ]:
model = YOLO("best.pt")

In [ ]:
def calculate_distance(focal_length, object_width, pixel_width):
    distance = (focal_length * object_width) / pixel_width
    return distance

def focal_length_finder (measured_distance, real_width, width_in_rf):
    focal_length = (width_in_rf * measured_distance) / real_width

    return focal_length

In [ ]:

def main():
    
    # box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=2, text_scale=1)
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        results=model.predict(frame,show = True)

        for r in results:
            annotator = Annotator(frame)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
                c = box.cls
                w = box.xywh[0][2]
                h = box.xywh[0][3]
                
                # fl = focal_length_finder(30,20,w)
                # print("focal length = ",fl)

                distance = calculate_distance(800, 20, w)
                # print("distance = ",distance)
                # convert the distance from tensor to float
                


                # #get the width and height of the box
                # print("width = ", b[2] - b[0])
                # print("height =", b[3] - b[1])
                # print("width = ", box.xywh[0][2])
                # print("height = ", box.xywh[0][3])
                # print(c) #class
                # print(model.names[int(c)]) #class name
                # print(box) #box object
                # print(box.xyxy[0]) #box coordinates
                # print(box.conf) #confidence

                annotator.box_label(b, model.names[int(c)] + " " + str(distance.item()))
#                 annotator.box_label(b, model.names[int(c)])

        frame = annotator.result()  

        cv2.imshow('YOLO', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()